In [ ]:
pip install yfinance

In [1]:
import numpy as np
import pandas as pd
import datetime
import yfinance as yf

### Tesla Stock Price, NASDAQ Composite, S&P 500 and Oil price

In [2]:
tesla_idx = "TSLA"
NASDAQ_composite_idx = '^IXIC'
SnP_500_idx = '^GSPC'
oil_price_idx = 'AAPL'

start_date = "2021-01-01"
end_date = "2022-12-30" # Il n'y a pas les données pour le 31 décembre 2022, le 01 et 02 janvier 2023

tesla_stock_price = yf.download(tesla_idx, start=start_date, end=end_date)
NASDAQ_composite = yf.download(NASDAQ_composite_idx, start=start_date, end=end_date)
SnP_500 = yf.download(SnP_500_idx, start=start_date, end=end_date)
oil_price = yf.download(oil_price_idx, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
# Création d'un array avec toutes les dates du 3 janvier 2021 au 4 janvier 2023
date_debut = datetime.datetime(2021, 1, 1)
date_fin = datetime.datetime(2022, 12, 30)
dates = np.arange(date_debut, date_fin, datetime.timedelta(days=1))

# Récupération des jours d'ouvertures du marché
market_days = tesla_stock_price.index.to_numpy()

# Récupération des valeurs en bourses à l'ouverture de nos paramètres
tesla_open = tesla_stock_price['Open'].to_numpy()
NASDAQ_open = NASDAQ_composite['Open'].to_numpy()
SnP_500_open = SnP_500['Open'].to_numpy()
oil_open = oil_price['Open'].to_numpy()


### Tesla deaths cases

In [4]:
df_deaths = pd.read_excel("../Datasets/Tesla Deaths.xlsx", header = 1)

deaths_dates = df_deaths['Date'].to_numpy()
deaths_count = df_deaths['Deaths'].to_numpy(dtype=int)


### Tweets of Elon Musk

In [5]:
df_tweets_2021 = pd.read_csv("../Datasets/tweets_2021.csv")[::-1]
df_tweets_2022 = pd.read_csv("../Datasets/Elon musk tweets 2022.csv")[::-1]
df_tweets = pd.concat([df_tweets_2021, df_tweets_2022]).reset_index(drop = True)

tweets_dates = np.array([np.datetime64(date.split(' ')[0]) for date in df_tweets['date'].to_numpy()], dtype='datetime64[us]')
tweets_content = df_tweets['Content'].to_numpy()
tweets_likes = df_tweets['Likes'].to_numpy()
tweets_retweet = df_tweets['Retweet'].to_numpy()


### Installation des bornes Tesla

In [6]:
df_bornes = pd.read_csv("../Datasets/bornes_tesla.csv", dtype = object)

bornes_dates = np.array([np.datetime64(date.split('T')[0]) for date in df_bornes['created_at'].to_numpy()], dtype='datetime64[us]')


### Mise en forme de toutes les colonnes en format np.array

In [7]:
# Tous les lags qui seront utilisés dans le DataFrame final
lag1 = 2
lag3 = 4
lag7 = 8
lag30 = 31
nb_days = len(dates)

# Voici la liste de tous les paramètres
parametres = ['Date', 'Tesla Stock Price', 'NASDAQ Composite', 'S&P 500', 'Oil Price', 
              'Deaths', 'Deaths Lag '+str(lag1-1), 'Deaths Lag '+str(lag7-1), 'Deaths Lag '+str(lag30-1), 
              'Tweets of Elon Musk', 'Tweet with mention Tesla','% Tweet w/ mention Tesla', 
              'Tweet w/ mention Tesla Lag '+str(lag1-1), 'Tweet w/ mention Tesla Lag '+str(lag3-1), 
              'Tweet w/ mention Tesla Lag '+str(lag7-1), 'Tweet Likes', 'Tweet Likes Lag '+str(lag1-1),
              'Tweet Likes Lag '+str(lag3-1), 'Tweet Likes Lag '+str(lag7-1), 'Retweet', 
              'Retweet Lag '+str(lag1-1), 'Retweet Lag '+str(lag3-1),  'Retweet Lag '+str(lag7-1), 
              'Installation bornes de recharge', 'Bornes Lag '+str(lag7-1), 'Bornes Lag '+str(lag30-1) ]


In [8]:
len(parametres)

26

#### Variable 1 : Dates

In [9]:
dates_format_df = np.array([np.datetime_as_string(dates[i], unit='D') for i in range(nb_days)])

In [10]:
dates_format_df[0]

'2021-01-01'

#### Variables 1 à 5 : Date, Tesla Stock, NASDAQ Composite, S&P 500, Prix du pétrole

In [11]:
market_nb_para = 5

data_market = np.zeros([nb_days,market_nb_para], dtype = object)
count_market_day = 0 # Cette variable nous permet de suivre les jours d'ouverture du marché

for i in range(nb_days):
    
    #Ajout dates
    data_market[i,0] = dates_format_df[i]
    # Ajout valeurs sur le marché
    data_market[i,1] = tesla_open[count_market_day]
    data_market[i,2] = NASDAQ_open[count_market_day]
    data_market[i,3] = SnP_500_open[count_market_day]
    data_market[i,4] = oil_open[count_market_day]
    # On suit les jours d'ouverture du marché
    if dates[i] == market_days[count_market_day]:
        count_market_day += 1

In [12]:
data_market[10]

array(['2021-01-11', 283.1333312988281, 13048.7802734375,
       3803.139892578125, 129.19000244140625], dtype=object)

#### Variables 6 et 7 : Nombre de morts, avec lag de 1 jour, 7 jours et 30 jours

In [13]:
deaths_nb_para = 4

data_deaths = np.zeros([nb_days,deaths_nb_para], dtype = int)

for i in range(nb_days):
    
    # Ajout des morts, on cherche s'il y a eu un déces à la dates[i] grâce à notre liste des dates de décès
    idx_death = np.where(deaths_dates == dates[i])[0]
    if len(idx_death) > 0:
        for j in idx_death:
            # Mort sans lag
            data_deaths[i,0] = deaths_count[j]
            
            # Mort avec lag1
            if nb_days-i >= lag1:
                data_deaths[i:i+lag1,1] += deaths_count[j]
            else:
                data_deaths[i:,1] += deaths_count[j]
                
            # Mort avec lag7
            if nb_days-i >= lag7:
                data_deaths[i:i+lag7,2] += deaths_count[j]
            else:
                data_deaths[i:,2] += deaths_count[j]
                
            # Mort avec lag30
            if nb_days-i >= lag30:
                data_deaths[i:i+lag30,3] += deaths_count[j]
            else:
                data_deaths[i:,3] += deaths_count[j]

In [14]:
data_deaths[9]

array([2, 2, 2, 2])

#### Variables 8 à 13 : Nombre de tweets; Nombre avec mention Tesla; Proportion avec mention Tesla; Tweets Tesla lag 1 jour, 3 jours, 1 semaine; Nombre de likes, avec/sans lags; Nombre de retweets, avec/sans lags

In [15]:
tweets_nb_para = 14

data_tweets = np.zeros([nb_days,tweets_nb_para])

for i in range(nb_days):
    
    # Ajout de la date des tweets, likes et retweet, ainsi que la mention du mot Tesla
    idx_tweet = np.where(tweets_dates == dates[i])[0]
    nb_tweets = len(idx_tweet)
    if nb_tweets > 0:
        # On ajoute le nombre de tweets
        data_tweets[i,0] = nb_tweets
        
        for j in idx_tweet:
            # On regarde si le tweet contient le mot tesla
            if 'Tesla' in tweets_content[j] or 'tesla' in tweets_content[j]:
                data_tweets[i,1] += 1
                # On ajoute le nombre de tweets avec lag1
                if nb_days-i >= lag1:
                    data_tweets[i:i+lag1,3] += 1
                else:
                    data_tweets[i:,3] += 1
                # On ajoute le nombre de tweets avec lag3
                if nb_days-i >= lag3:
                    data_tweets[i:i+lag3,4] += 1
                else:
                    data_tweets[i:,4] += 1
                # On ajoute le nombre de tweets avec lag7
                if nb_days-i >= lag7:
                    data_tweets[i:i+lag7,5] += 1
                else:
                    data_tweets[i:,5] += 1
            
            # Nombre de likes
            data_tweets[i,6] = tweets_likes[j]
            if nb_days-i >= lag1:
                data_tweets[i:i+lag1,7] += tweets_likes[j]
            else:
                data_tweets[i:,7] += tweets_likes[j]
            # On ajoute le nombre de tweets avec lag3
            if nb_days-i >= lag3:
                data_tweets[i:i+lag3,8] += tweets_likes[j]
            else:
                data_tweets[i:,8] += tweets_likes[j]
            # On ajoute le nombre de tweets avec lag7
            if nb_days-i >= lag7:
                data_tweets[i:i+lag7,9] += tweets_likes[j]
            else:
                data_tweets[i:,9] += tweets_likes[j]
            
            # Nombre de retweet
            data_tweets[i,10] += tweets_retweet[j]
            if nb_days-i >= lag1:
                data_tweets[i:i+lag1,11] += tweets_retweet[j]
            else:
                data_tweets[i:,11] += tweets_retweet[j]
            # On ajoute le nombre de tweets avec lag3
            if nb_days-i >= lag3:
                data_tweets[i:i+lag3,12] += tweets_retweet[j]
            else:
                data_tweets[i:,12] += tweets_retweet[j]
            # On ajoute le nombre de tweets avec lag7
            if nb_days-i >= lag7:
                data_tweets[i:i+lag7,13] += tweets_retweet[j]
            else:
                data_tweets[i:,13] += tweets_retweet[j]
        
        # On ajoute la proportions de mention Tesla
        data_tweets[i,2] = data_tweets[i,1]/data_tweets[i,0]

In [16]:
data_tweets[-8]

array([2.0000000e+01, 2.0000000e+00, 1.0000000e-01, 3.0000000e+00,
       3.0000000e+00, 9.0000000e+00, 8.5350000e+03, 3.9889190e+06,
       7.9671530e+06, 2.3174449e+07, 1.5907200e+05, 3.3531900e+05,
       8.3160400e+05, 2.2149140e+06])

#### Variable 14 : Installation des bornes de recharges, avec lag 7 jours et 30 jours

In [17]:
bornes_nb_para = 3

data_bornes = np.zeros([nb_days, bornes_nb_para], dtype = int)

for i in range(nb_days):
    
    # Même procédé que pour le nombre de tweets
    # Ajout de la date d'installation des bornes
    idx_bornes = np.where(bornes_dates == dates[i])[0]
    nb_bornes = int(len(idx_bornes))
    if nb_bornes > 0:
        # On ajoute le nombre de bornes installées à la date i
        data_bornes[i,0] = nb_bornes
        
        # On ajoute le nombre de bornes installées avec lag7
        if nb_days-i >= lag7:
            data_bornes[i:i+lag7,1] += nb_bornes
        else:
            data_bornes[i:,1] += nb_bornes
        # On ajoute le nombre de bornes installées avec lag30
        if nb_days-i >= lag30:
            data_bornes[i:i+lag30,2] += nb_bornes
        else:
            data_bornes[i:,2] += nb_bornes


In [18]:
data_bornes[200]

array([  0,   4, 276])

### Mise en DataFrame

In [19]:
all_data = np.concatenate((data_market, data_deaths, data_tweets, data_bornes), axis = 1)

df = pd.DataFrame (all_data, columns = parametres)

In [20]:
df.tail(10)

,Date,Tesla Stock Price,NASDAQ Composite,S&P 500,Oil Price,Deaths,Deaths Lag 1,Deaths Lag 7,Deaths Lag 30,Tweets of Elon Musk,...,Tweet Likes Lag 1,Tweet Likes Lag 3,Tweet Likes Lag 7,Retweet,Retweet Lag 1,Retweet Lag 3,Retweet Lag 7,Installation bornes de recharge,Bornes Lag 7,Bornes Lag 30
718,2022-12-20,146.050003,10490.889648,3810.469971,131.389999,0,1,3,10,39.0,...,3978234.0,12537622.0,22669482.0,346764.0,496285.0,1294065.0,2127589.0,0,36,73
719,2022-12-21,139.339996,10592.0,3839.48999,132.979996,0,0,3,10,47.0,...,4470898.0,12448377.0,22504897.0,176247.0,523011.0,1339396.0,2152394.0,0,21,73
720,2022-12-22,136.0,10586.459961,3853.26001,134.350006,1,1,4,11,20.0,...,3988919.0,7967153.0,23174449.0,159072.0,335319.0,831604.0,2214914.0,0,21,60
721,2022-12-23,126.370003,10437.75,3815.110107,130.919998,0,1,4,11,35.0,...,3962339.0,8433237.0,23866793.0,98181.0,257253.0,780264.0,2194729.0,2,23,61
722,2022-12-24,117.5,10462.19043,3843.340088,131.380005,0,0,4,11,22.0,...,2829553.0,6818472.0,19356094.0,65769.0,163950.0,499269.0,1793334.0,0,23,60
723,2022-12-25,117.5,10462.19043,3843.340088,131.380005,0,0,3,11,6.0,...,1577892.0,5540231.0,17988608.0,34750.0,100519.0,357772.0,1697168.0,0,23,58
724,2022-12-26,117.5,10462.19043,3843.340088,131.380005,0,0,2,11,19.0,...,1336681.0,4166234.0,12133387.0,83065.0,117815.0,281765.0,1113369.0,0,23,58
725,2022-12-27,117.5,10462.19043,3843.340088,131.380005,0,0,1,10,20.0,...,2504239.0,4082131.0,12515368.0,144205.0,227270.0,327789.0,1108053.0,272,274,330
726,2022-12-28,110.349998,10339.200195,3829.560059,129.669998,0,0,1,9,49.0,...,6362302.0,7698983.0,14517455.0,561140.0,705345.0,823160.0,1322429.0,0,274,330
727,2022-12-29,120.389999,10321.459961,3805.449951,127.989998,0,0,1,7,9.0,...,5142266.0,7646505.0,13186736.0,30952.0,592092.0,819362.0,1177134.0,377,651,705


In [22]:
df.to_excel('DataTesla.xlsx', index = False)


In [23]:
df.to_csv('DataTesla.csv', index = False)